# Swap on Curve

> CRYPTO | 250
> 
> One day, I tried to swap x and y coordinates of a Point on the Curve.
>
> Attachment: swap_on_curve.tar.gz (md5 8f4f898bdeafd570c2380396b2515c12)

This is a sagemath notebook

- Open `task.sage`

```python
from params import p, a, b, flag, y

x = int.from_bytes(flag, "big")

assert 0 < x < p
assert 0 < y < p
assert x != y

EC = EllipticCurve(GF(p), [a, b])

assert EC(x,y)
assert EC(y,x)

print("p = {}".format(p))
print("a = {}".format(a))
print("b = {}".format(b))
```

- We are given parameters for an elliptic curve `p`, `a` and `b`
- There are two numbers `x` and `y` such that both `(x, y)` and `(y, x)` are on the curve:
$$
\begin{align*}
y^2 & = x^3 + a x + b \mod p \\
x^2 & = y^3 + a y + b \mod p
\end{align*}
$$
- We need to solve for both `x` and `y`

- Rearrange to get polynomial only in `x`:
$$
\begin{align*}
y & = \sqrt{x^3 + a x + b} \mod p\\
x^2 & = (\sqrt{x^3 + a x + b})^3 + a \sqrt{x^3 + a x + b} + b \mod p & \text{(Take square root)} \\
& = (x^3 + a x + b) \cdot \sqrt{x^3 + a x + b} + a \sqrt{x^3 + a x + b} + b \mod p \\
& = (x^3 + a x + b + a) \cdot \sqrt{x^3 + a x + b} + b \mod p \\
x^2 - b & = (x^3 + a x + b + a) \cdot \sqrt{x^3 + a x + b} \mod p  & \text{(Rearrange } b \text{)} \\
(x^2 - b)^2 & = (x^3 + a x + b + a)^2 \cdot (x^3 + a x + b) \mod p  & \text{(Square both sides)} \\
0 & = (x^3 + a x + b + a)^2 \cdot (x^3 + a x + b) - (x^2 - b)^2 \mod p  & \text{(Rearrange)} 
\end{align*}
$$
- Solve for `x` using sagemath, then we can solve for `y` too

In [1]:
p = 10224339405907703092027271021531545025590069329651203467716750905186360905870976608482239954157859974243721027388367833391620238905205324488863654155905507
a = 4497571717921592398955060922592201381291364158316041225609739861880668012419104521771916052114951221663782888917019515720822797673629101617287519628798278
b = 1147822627440179166862874039888124662334972701778333205963385274435770863246836847305423006003688412952676893584685957117091707234660746455918810395379096

In [2]:
F.<x> = GF(p)[]
k = (x^3 + a * x + b)
fn = (k + a)^2 * (k) - (x^2 - b)^2
fn = fn.monic()
fn

x^9 + 3268375747857074104837911746245059118284023145296920209112468680455643131386336956833508202186993690747627639362690713770848154115681980362998904730489327*x^7 + 2214271912256019206471472943317231723997577092315878601392884641882287708707742976977861168083107707941875431199729069401300478146035118113467816287828337*x^6 + 3846076180419448177395463688731587357824551587191602571783361346273393917583357054155715813917840847867366863694458946821285438088046312565285184110470994*x^5 + 8448522451907471754391938946799218455891276427716845038821936957372640022952184725873747626168477964929811231597249483489073177911730517387751843206482339*x^4 + 531114221440585523332727843591364887935896045398105264357837053276299802562148375308829360072747096952151783544292121675302581713977646678427973104465850*x^3 + 4274806073170927375980680427648224874571435846402103777667656950196483722802668587089436936984333307713479821988986282130131268462697970991760106508737672*x^2 + 62845491397507057962624297851

In [3]:
res = fn.roots()
res

[(7701093676539600849545233775656266124024393901117284843908110961515787578426141185510163529087490707005602090170967486760425035325079989548242482516345996,
  1),
 (7677785116435727686649964446582280200653867847347508329665823423662012251739816682685754769395346602260506848926697976097716406737781893094340049957504162,
  1),
 (3418100096957777329641522385874601707957378769808441636922695882579741403099793135186593317827442718653892790484853393209667843889446155946725442732255448,
  1)]

- Just print all solutions, one must be the flag!

In [4]:
for x, _ in res:
  print(bytes.fromhex(hex(int(x))[2:]))

b'\x93\n)jF\x82K\xabgIM\xbc\xfeT\x8c\xf8&b\xc4\xd7\xb08?\xf8\xfb\xa7\xb2\x8e.\xf32\xffKIX\x0e\\\xf3Bq\x9f\xd3\x97\x922H\xa1\xe0\xeblE\xa5\x13\xaf\x06\xd8t\x84\x834\xd2\xdeD\x8c'
b'\x92\x98;^i\t\x8a\xe2h1\xae\xd0YYYXEV\x02ZKp\xc5\x05\xabF\xe1\x98\x83\x92\x80\xc2f\xf6\x7f\xe0\x96\x84jN\x18\xa4\xa1\x92\xbe\x98!\x95o\xd5\x1f\xc46OyrVq\x89G\x14\xc3D\xa2'
b'ACSC{have_you_already_read_the_swap<-->swap?})\xd6\x82a\x076s;\x1e\xaf\x13\x92\x1f)\x997-h\xd8'


- Sanity check that the coordinates are indeed swappable

In [5]:
EC = EllipticCurve(GF(p), [a, b])
found = False
for x, _ in res:
  ys = EC.lift_x(x, all=True)
  for y in ys:
    if x == y[1]:
      continue
    zs = EC.lift_x(y[1], all=True)
    for z in zs:
      if int(x) == int(z[1]):
        print(x, y[1])
        found = True
        break
  if found: break

7677785116435727686649964446582280200653867847347508329665823423662012251739816682685754769395346602260506848926697976097716406737781893094340049957504162 3418100096957777329641522385874601707957378769808441636922695882579741403099793135186593317827442718653892790484853393209667843889446155946725442732255448


Flag: `ACSC{have_you_already_read_the_swap<-->swap?}`